# Preprocessing Shoe Image Data for Use in Drive Data Generator

### Create Feature Dataframe with Local Image Paths

In [1]:
# Create a dataframe with meta_info and drive, already done via munger
from munger import Munger
mngr = Munger()
df_meta = mngr.munge()[0]

Returning meta_data dataframe followed by feature_space_dataframe for  data/4096_vals.csv


In [2]:
df_meta.head()

,CID,Category,SubCategory,HeelHeight,Insole,Closure,Gender,Material,ToeStyle,img_path
CID,,,,,,,,,,
100627-72,100627-72,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...
100627-255,100627-255,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...
100657-72,100657-72,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...
100657-216,100657-216,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...
101026-3,101026-3,Boots,Mid-Calf,1in - 1 3/4in,Leather;Padded,Pull-on,Men,Leather;Rubber,Square Toe;Closed Toe,data/ut-zap50k-images/Boots/Mid-Calf/Durango/1...


## Preprocess Images and Save to a Local Preprocessed Folder

### Define preprocessing Functions here

In [3]:
#img preprocessing functions
def convert_img(img, conversion_str):
    return img.convert(conversion_str)

def resize_img(img, resize):
    return img.resize(resize)

grayscale_img = lambda img: convert_img(img, 'L')
resize_120 = lambda img: resize_img(img, (120, 120))

In [4]:
#pick 120 as a nice even number between 106 and 132 (their mean is 119)

preprocesses = [grayscale_img, resize_120]

In [5]:
from PIL import Image

def preprocess_img_row(row, folder_path='pp_data', preprocesses=[]):
    with Image.open(row['img_path']) as img:
        #Preprocess image
        for preprocess in preprocesses:
            img = preprocess(img)
        
        #Save preprocessed image
        preprocessed_path = 'images/' + row['CID'] + '.jpg'
        save_path = folder_path + '/' + preprocessed_path
        img.save(save_path)
    row['path'] = preprocessed_path
    return row    

### Define name of preprocessed data folder here

In [6]:
preprocessed_path = 'pp_data'

### Perform Preprocessing!

In [7]:
df_preprocessed = df_meta.apply(lambda row: preprocess_img_row(row, folder_path=preprocessed_path, preprocesses=preprocesses), axis=1)

In [8]:
df_preprocessed.head()

,CID,Category,SubCategory,HeelHeight,Insole,Closure,Gender,Material,ToeStyle,img_path,path
CID,,,,,,,,,,,
100627-72,100627-72,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...,images/100627-72.jpg
100627-255,100627-255,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...,images/100627-255.jpg
100657-72,100657-72,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...,images/100657-72.jpg
100657-216,100657-216,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,data/ut-zap50k-images/Shoes/Oxfords/Bostonian/...,images/100657-216.jpg
101026-3,101026-3,Boots,Mid-Calf,1in - 1 3/4in,Leather;Padded,Pull-on,Men,Leather;Rubber,Square Toe;Closed Toe,data/ut-zap50k-images/Boots/Mid-Calf/Durango/1...,images/101026-3.jpg


### Save CSV in Preprocessed folder!

In [9]:
#create and save drive_csv
import pandas
df_preprocessed.drop(['CID', 'img_path'], axis=1, inplace=True)

csv_path = preprocessed_path + '/df.csv'
df_preprocessed.to_csv(csv_path)

## Test that we can read in csv correctly!

In [10]:
drive_df_test = pandas.read_csv(csv_path, index_col=0)

In [11]:
drive_df_test.head()

,Category,SubCategory,HeelHeight,Insole,Closure,Gender,Material,ToeStyle,path
CID,,,,,,,,,
100627-72,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,images/100627-72.jpg
100627-255,Shoes,Oxfords,NaN,Leather,Lace up,Men,Leather,Capped Toe;Round Toe,images/100627-255.jpg
100657-72,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,images/100657-72.jpg
100657-216,Shoes,Oxfords,NaN,Leather;Padded;Removable,Lace up,Men,Leather,Capped Toe;Round Toe,images/100657-216.jpg
101026-3,Boots,Mid-Calf,1in - 1 3/4in,Leather;Padded,Pull-on,Men,Leather;Rubber,Square Toe;Closed Toe,images/101026-3.jpg


## Upload Image Files from Local Preprocessed Folder to Drive

#CONNECT TO DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

def upload_img_row(row, drive):
    f = drive.CreateFile()
    f.SetContentFile(row['preprocessed_path'])
    f.Upload()
    row['drive_id'] = f['id']
    return row

from IPython.display import clear_output

def upload_to_drive(df, drive):
    '''Takes in the preprocessed_dataframe and uploads local
    stored preprocessed images to drive, returning a dataframe
    with drive file_ids corresponding to correct image attached 
    to each data item'''
    count = 0
    length = len(df)
    drive_df = pandas.DataFrame()
    for index, row in df.iterrows():
        count += 1
        
        drive_df.append(upload_img_row(row, drive))
        clear_output()
        print('{}%'.format(round(100 * (count/length), 2)))
    return drive_df
        

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

for file in file_list:
    if file['title'].startswith('pp_data'):
        file.Delete()